# VM 使用 LLaMA-Factory Docker 部署 Llama3.1-8B 微调模型

本 Jupyter Notebook 提供了在虚拟机（VM）上使用 LLaMA-Factory Docker 方式部署 Llama3.1-8B 模型的完整指南。本教程涵盖了从环境配置到模型部署和测试的所有步骤。

## 概述

**LLaMA-Factory** 是一个开源的大语言模型微调框架，提供了简单易用的工具来对 LLaMA、ChatGLM、Baichuan 等主流大模型进行高效的参数微调和训练

**Llama3.1-8B** 是 Meta 发布的一个高效的大型语言模型，本教程将指导您如何在虚拟机（VM）上完整部署 Llama3.1-8B 模型，包括环境配置、模型下载、部署和测试等步骤。

**Docker** 是一个容器化平台，将应用程序及其依赖环境打包成轻量级、可移植的容器，实现"一次构建，处处运行"的部署方式。

**Azure Kubernetes Service (AKS)** 是微软 Azure 提供的托管 Kubernetes 服务，简化了容器化应用的部署、管理和扩展，无需用户管理 Kubernetes 控制平面。

## 环境要求

### 硬件要求
- **CPU**: 至少 8 核心（推荐 16 核心或以上）
- **内存**: 最少 16GB RAM（推荐 32GB 或以上）
- **存储**: 至少 50GB 可用空间
- **GPU**: NVIDIA GPU（至少 16GB 显存）

### 软件要求
- **操作系统**: Ubuntu 20.04/22.04 LTS 或 CentOS 7/8
- **Python**: 3.8 或更高版本
- **Docker**: 20.10 或更高版本
- **CUDA**: 11.8 或更高版本（如果使用 GPU）

### 系统要求
- Azure 订阅（如果没有，请创建[免费账户](https://azure.microsoft.com/free/)）
- Azure CLI 2.50.0 或更高版本
- Docker 命令行工具
- 至少 16GB RAM 的开发机器

## 环境准备

### 定义环境变量

我们需要设置一些环境变量来管理我们的资源：

In [ ]:
import os

# 设置环境变量
os.environ["RESOURCE_GROUP"] = "lenovo_vm_rg"
os.environ["LOCATION"] = "eastus2"
os.environ['VM_NAME'] = "llama3_1_8b-vm"
os.environ['VM_SIZE'] = "Standard_D4s_v3"

# 显示配置
print(f"Resource Group: {os.environ['RESOURCE_GROUP']}")
print(f"Location: {os.environ['LOCATION']}")
print(f"VM Name: {os.environ['VM_NAME']}")
print(f"VM Size: {os.environ['VM_SIZE']}")


## VM 环境准备

### 1. 创建资源组

Azure 中的资源组是一个逻辑容器, 它允许您根据资源的生命周期和安全需求将这些资源作为一个整体进行管理。资源组中的资源可以包括虚拟机、存储账户和虚拟网络等。

以下使用 Azure CLI 创建 VM（需要先安装和配置 Azure CLI）：

In [ ]:
%%bash
az group create \
    --name $RESOURCE_GROUP \
    --location $LOCATION

### 2. 创建 VM

以下使用 Azure CLI 创建 VM（需要先安装和配置 Azure CLI）：

In [ ]:
# 使用 Azure CLI 创建 VM
!az vm create \
  --resource-group $RESOURCE_GROUP \
  --name $VM_NAME \
  --image Ubuntu2204 \
  --size $VM_SIZE \
  --admin-username azureuser \
  --generate-ssh-keys

### 2. 连接到 VM

获取 VM 的 IP 地址后，使用 SSH 连接：

In [ ]:
# 替换为您的 VM IP 地址
VM_IP="<VM-IP-ADDRESS>"
!ssh azureuser@$VM_IP

> 下面的操作都将在虚拟机上进行，请将对应的命令拷贝到虚拟机上执行

### 3. 更新系统

运行以下命令更新系统并安装基础工具：

In [ ]:
# 更新包列表和系统
!sudo apt update && sudo apt upgrade -y

# 安装基础工具
!sudo apt install -y build-essential git wget curl

## 安装依赖

### 1. 安装 Docker 和 Docker Compose

In [ ]:
# 安装 Docker
!curl -fsSL https://get.docker.com -o get-docker.sh
!sudo sh get-docker.sh

# 将当前用户添加到 docker 组
!sudo usermod -aG docker $USER

# 安装 Docker Compose
!sudo curl -L "https://github.com/docker/compose/releases/latest/download/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
!sudo chmod +x /usr/local/bin/docker-compose

# 启动 Docker 服务
!sudo systemctl start docker
!sudo systemctl enable docker

# 验证安装
!docker --version
!docker-compose --version

### 2. 安装 NVIDIA Container Toolkit（如果使用 GPU）

In [ ]:
# 添加 NVIDIA Docker 仓库
!curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg
!curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
  sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
  sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list

# 更新包列表并安装
!sudo apt-get update
!sudo apt-get install -y nvidia-container-toolkit

# 配置 Docker 以使用 NVIDIA runtime
!sudo nvidia-ctk runtime configure --runtime=docker
!sudo systemctl restart docker

# 验证 GPU 支持
!docker run --rm --gpus all nvidia/cuda:11.8-base-ubuntu22.04 nvidia-smi

### 3. 克隆 LLaMA-Factory 项目

In [ ]:
# 克隆 LLaMA-Factory 项目
!git clone https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory

# 查看项目结构
!ls -la LLaMA-Factory/

## 部署模型

### 1. 构建 Docker 镜像

使用 `llama-factory` 官方 docker-compose 脚本启动服务

In [ ]:
# 进入 LLaMA-Factory 目录
%cd LLaMA-Factory/docker/docker-cuda/

# 启动服务
!docker compose up -d

# 测试容器服务
!docker compose exec llamafactory bash

### 2. 导入微调模型

`Dockerfile` 通过 VOLUME 描述挂载目录，

```yaml
VOLUME [ "/root/.cache/huggingface", "/root/.cache/modelscope", "/app/data", "/app/output" ]
```

- `/root/.cache/huggingface`：huggingface 模型缓存目录
- `/root/.cache/modelscope`： modelscope 模型缓存目录

用户根据需要将微调模型放入到对应目录当中

### 3. 创建 API 服务启动脚本

创建一个脚本来启动 LLaMA-Factory 的 API 服务：

In [ ]:
%%writefile start_llama3_api.sh
#!/bin/bash

# 启动 LLaMA-Factory API 服务
docker exec -d llamafactory llamafactory-cli api \
    examples/inference/llama3.yaml \
    infer_backend=vllm \
    vllm_enforce_eager=true \
    --host 0.0.0.0 \
    --port 8000

echo "Llama3 API 服务已启动在端口 8000"
echo "访问 http://localhost:8000/docs 查看 API 文档"

In [ ]:
# 给脚本添加执行权限
!chmod +x start_api.sh

# 启动 API 服务
!./start_api.sh

## 测试验证

### 1. 检查容器状态

In [ ]:
# 检查容器运行状态
!docker ps

# 查看容器日志
!docker logs llamafactory

# 检查 GPU 使用情况（如果有 GPU）
!docker exec llamafactory nvidia-smi

### 2. 测试 API 接口

In [ ]:
# 测试健康检查
!curl http://localhost:8000/v1/models

# 测试聊天接口
%%bash
curl -X POST "http://localhost:8000/v1/chat/completions" \
     -H "Content-Type: application/json" \
     -d '{
       "model": "microsoft/Llama3_1_9b",
       "messages": [
         {
           "role": "user",
           "content": "什么是人工智能？"
         }
       ],
       "max_tokens": 512,
       "temperature": 0.7
     }'

### 3. 监控和日志

In [ ]:
# 监控容器资源使用情况
!docker stats llamafactory-phi4

# 查看详细的容器信息
!docker inspect llamafactory-phi4

# 实时查看日志
!docker logs -f llamafactory-phi4

## 常见问题

### 1. Docker 容器内存不足

In [ ]:
# 增加 Docker 容器的内存限制
!docker update --memory=32g --memory-swap=64g llamafactory

# 或者在 docker-compose.yml 中设置
# deploy:
#   resources:
#     limits:
#       memory: 32G

### 2. GPU 在容器中不可用

In [ ]:
# 检查 NVIDIA Container Toolkit 是否正确安装
!docker run --rm --gpus all nvidia/cuda:11.8-base-ubuntu22.04 nvidia-smi

# 重新配置 Docker 以支持 GPU
!sudo nvidia-ctk runtime configure --runtime=docker
!sudo systemctl restart docker

# 确保 docker-compose.yml 中包含 GPU 配置
# deploy:
#   resources:
#     reservations:
#       devices:
#         - driver: nvidia
#           count: all
#           capabilities: [gpu]

In [ ]:
# 在容器内设置 Hugging Face 镜像
!docker exec llamafactory bash -c "export HF_ENDPOINT=https://hf-mirror.com"

# 或者在 docker-compose.yml 中设置环境变量
# environment:
#   - HF_ENDPOINT=https://hf-mirror.com

# 使用代理下载（如果需要）
# environment:
#   - https_proxy=http://your-proxy:port
#   - http_proxy=http://your-proxy:port